Blood culture test prediction model using medical history

In [1]:
!pip install swifter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.2/242.2 kB 18.7 MB/s eta 0:00:00
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16506 sha256=fb257e3c7c6aeba1e031af16f9ad484678716e4fb05f86bc9bd9a7610187617f
  Stored in directory: /root/.cache/pip/wheels/e4/cf/51/0904952972ee2c7aa3709437065278dc534ec1b8d2ad41b443
Successfully built swifter
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.6.4
    Uninstalling importlib-metadata-4.6.4:
      Successfully uninstalled importlib-metadata-4.6.4


In [2]:
import swifter

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import keras
import numpy as np
from tensorflow.keras.utils import pad_sequences

In [5]:
import tensorflow as tf
print(tf.__version__)

2.15.0


Uploading Medical Data

In [6]:
path ='/content/drive/MyDrive/BCTP/nimess_features.csv'
nimess_eventlog = pd.read_csv(path, low_memory=False)

In [7]:
path ='/content/drive/MyDrive/BCTP/anino_features.csv'
anino_eventlog = pd.read_csv(path, low_memory=False)

In [8]:
nsml_eventlog = pd.read_csv('/content/drive/MyDrive/BCTP/nsml_eventlog_aggregated.csv', low_memory=False)

In [9]:
path ='/content/drive/MyDrive/BCTP/trfl_eventlog.csv'
trfl_eventlog = pd.read_csv(path, low_memory=False)

In [10]:
path ='/content/drive/MyDrive/BCTP/xgboost_data.csv'
final_df = pd.read_csv(path, low_memory=False)

Data cleaning and preprocessing

In [11]:
nimess_eventlog.drop(['Unnamed: 0'], axis=1, inplace=True)
anino_eventlog.drop(['Unnamed: 0'], axis=1, inplace=True)
nsml_eventlog.drop(['Unnamed: 0'], axis=1, inplace=True)
trfl_eventlog.drop(['Unnamed: 0'], axis=1, inplace=True)
final_df.drop(['Unnamed: 0'], axis=1, inplace=True)

nimess_eventlog['date_column'] = pd.to_datetime(nimess_eventlog['date_column'])
anino_eventlog['date_column'] = pd.to_datetime(anino_eventlog['date_column'])
nsml_eventlog['date_column'] = pd.to_datetime(nsml_eventlog['date_column'])
trfl_eventlog['date_column'] = pd.to_datetime(trfl_eventlog['date_column'])
final_df['date_column'] = pd.to_datetime(final_df['date_column'])

nimess_eventlog['Gender'] = nimess_eventlog['kjønn'].map({'Mann': 1, 'Kvinne': 0})

nimess_eventlog.drop(['index', 'disease_groups', 'kjønn', 'prior_comorbidities', 'inndatotid', 'utdatotid'], axis=1, inplace=True)

In [12]:
# Calculate the number of rows for each 'ppid'
ppid_row_counts = nimess_eventlog['ppid'].value_counts()

# Bin these counts into specified ranges
bins = [0, 3, 6, 301, 601, 1001, float('inf')]  # Adjusted bins to include the edge values
labels = ['0-2', '3-5', '6-300', '301-600', '601-1000', '1000+']
binned_counts = pd.cut(ppid_row_counts, bins=bins, labels=labels, right=False).value_counts().sort_index()

# Display the binned counts to verify
print(binned_counts)


count
0-2          2777
3-5          3497
6-300       29082
301-600       179
601-1000       42
1000+          14
Name: count, dtype: int64


In [13]:
# Calculate the number of rows for each 'ppid'
ppid_row_counts = trfl_eventlog['ppid'].value_counts()

# Bin these counts into specified ranges
bins = [0, 3, 6, 201, 301, 1001, float('inf')]  # Adjusted bins to include the edge values
labels = ['0-2', '3-5', '6-100', '201-300', '301-1000', '1000+']
binned_counts = pd.cut(ppid_row_counts, bins=bins, labels=labels, right=False).value_counts().sort_index()

# Display the binned counts to verify
print(binned_counts)


count
0-2          2473
3-5          4732
6-100       27122
201-300        93
301-1000       17
1000+           0
Name: count, dtype: int64


In [ ]:
anino_eventlog.columns

Index(['ppid', 'ICU_LOS', 'date_column', 'ICU_LOS_total'], dtype='object')

In [ ]:
trfl_eventlog.columns

Index(['ppid', 'BILIRUBIN KONJUGERT', 'BILIRUBIN TOTAL',
       'BILIRUBIN UKONJUGERT', 'CRP', 'CRP-HØYSENSITIV', 'KREATININ', 'LAKTAT',
       'LAKTAT BLODGASS', 'LAKTAT BLODGASS VENØST', 'LAKTAT PNA',
       'LEUKOCYTTER', 'PH', 'PH PNA', 'PO2', 'PO2 PNA', 'TROMBOCYTTER',
       'date_column'],
      dtype='object')

In [ ]:
nimess_eventlog.columns

Index(['ppid', 'urgency_code', 'care_level_code', 'LOS', 'time_to_last',
       'total_los', 'age', 'procedure_0', 'procedure_1', 'procedure_2',
       ...
       'procedure_R_aggregate', 'procedure_S_aggregate',
       'procedure_T_aggregate', 'procedure_U_aggregate',
       'procedure_V_aggregate', 'procedure_W_aggregate',
       'procedure_X_aggregate', 'procedure_Y_aggregate',
       'procedure_Z_aggregate', 'Gender'],
      dtype='object', length=167)

In [16]:
def encode_test_results(nsml_eventlog):
    test_result_columns = [col for col in nsml_eventlog.columns if col not in ['ppid', 'date_column']]

    for col in test_result_columns:
        # Ensure the column data is in string format
        nsml_eventlog[col] = nsml_eventlog[col].astype(str)

        nsml_eventlog[col] = nsml_eventlog[col].apply(
            lambda x: 5 if 'positive' in x else
                      2 if 'contaminant' in x and 'positive' not in x else
                      1 if 'negative' in x and 'positive' not in x and 'contaminant' not in x else
                      0  # Default value for empty or unspecified case
        )
    return nsml_eventlog

# Apply the updated encoding logic
encoded_nsml_eventlog = encode_test_results(nsml_eventlog.copy())


In [17]:
nimess_eventlog = nimess_eventlog.fillna(0)
anino_eventlog = anino_eventlog.fillna(0)
trfl_eventlog = trfl_eventlog.fillna(0)
encoded_nsml_eventlog = encoded_nsml_eventlog.fillna(0)

nimess_eventlog = nimess_eventlog[['ppid',
       'urgency_code', 'care_level_code', 'LOS',
       'time_to_last', 'total_los',
       'Gender', 'age', 'procedure_0', 'procedure_1',
       'procedure_3', 'procedure_4', 'procedure_A', 'procedure_B',
       'procedure_C', 'procedure_D', 'procedure_E', 'procedure_F',
       'procedure_G', 'procedure_H', 'procedure_I', 'procedure_J',
       'procedure_K', 'procedure_L', 'procedure_M', 'procedure_N',
       'procedure_O', 'procedure_P', 'procedure_Q', 'procedure_R',
       'procedure_T', 'procedure_U', 'procedure_W', 'procedure_X',
       'procedure_Z', 'ICD_A', 'ICD_B', 'ICD_C', 'ICD_D', 'ICD_E', 'ICD_F',
       'ICD_G', 'ICD_H', 'ICD_I', 'ICD_J', 'ICD_K', 'ICD_L', 'ICD_M', 'ICD_N',
       'ICD_O', 'ICD_Q', 'ICD_R', 'ICD_S', 'ICD_T', 'ICD_U', 'ICD_V', 'ICD_W',
       'ICD_X', 'ICD_Y', 'ICD_Z', 'date_column', 'urinarytractinfection', 'cardiovascular', 'lung', 'centralnervoussystem', 'organdysfunction', 'skinandsofttissueinfection',
                                   'pneumonia', 'endocarditis', 'sepsis', 'infection', 'dementia', 'explicitsepsis', 'intraabdominalinfection', 'cancer']]

final_df.drop(['date_column.2', 'ppid.2', 'date_column.1', 'ppid.1'], axis=1, inplace=True)

In [ ]:
encoded_nsml_eventlog

,ppid,date_column,annet,anus,bein,biopsi,blod,blood_culture_test,bronki,melk,...,faeces,hal,hud,led,naso,plasma,tunge,urin,ear,eye
0,1,2013-10-08,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1,2014-05-08,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,2014-08-31,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,2014-09-01,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,1,2014-12-29,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273468,36351,2017-06-01,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
273469,36351,2017-06-02,0,0,0,0,0,1,0,0,...,0,5,0,0,0,0,0,0,0,0
273470,36351,2017-06-07,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
273471,36351,2017-06-27,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [18]:
# Helper function to check if a date is within one week of a reference date
def is_within_one_week(date, reference_date):
    return abs((date - reference_date).days) <= 60

# List to hold the indices of rows to be removed
rows_to_remove = []

# Iterate through the DataFrame
for index, row in final_df.iterrows():
    if row['blood_culture_test_encoded'] == 1.0:  # Positive blood culture test
        ppid = row['ppid']
        positive_date = row['date_column']

        # Identify negative tests within one week before and after the positive test for the same ppid
        for idx, r in final_df[final_df['ppid'] == ppid].iterrows():
            if r['blood_culture_test_encoded'] == 0.0 and is_within_one_week(r['date_column'], positive_date):
                rows_to_remove.append(idx)

# Remove duplicate indices
rows_to_remove = list(set(rows_to_remove))

# Drop the identified rows
final_df = final_df.drop(rows_to_remove)

In [ ]:
final_df

,blood_culture_test_encoded,date_column,ppid,urgency_code,care_level_code,LOS,time_to_last,total_los,age,procedure_0,...,prior_positive_faeces,prior_positive_hal,prior_positive_hud,prior_positive_led,prior_positive_naso,prior_positive_plasma,prior_positive_tunge,prior_positive_urin,prior_positive_ear,prior_positive_eye
0,0.0,2014-05-08,1.0,1,1,5.458333,1524.0,22.958333,81,0,...,0,0,0,0,0,0,0,0,0,0
1,0.0,2014-08-31,1.0,1,1,4.958333,456.0,27.916667,81,0,...,0,0,0,0,0,0,0,0,0,0
2,0.0,2014-12-29,1.0,1,1,1.500000,838.0,29.416667,81,0,...,0,0,0,0,0,0,0,0,0,0
3,0.0,2013-10-28,2.0,1,1,3.000000,0.0,3.000000,44,0,...,0,0,0,0,0,0,0,0,0,0
4,0.0,2019-04-29,3.0,4,1,10.208333,65.0,131.166667,64,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65970,0.0,2017-05-30,36351.0,1,1,13.833333,9.0,31.750000,61,0,...,0,0,0,0,0,0,0,1,0,0
65971,0.0,2017-05-31,36351.0,1,1,13.833333,9.0,31.750000,61,0,...,0,0,0,0,0,0,0,1,0,0
65972,0.0,2017-06-01,36351.0,1,1,13.833333,9.0,31.750000,61,0,...,0,0,0,0,0,0,0,1,0,0
65973,0.0,2017-06-02,36351.0,1,1,13.833333,9.0,31.750000,61,0,...,0,0,0,0,0,0,0,1,0,0


For every blood culture test creating sequences of medical events leading upto it from the four eventlogs (nimess_eventlog, anino_eventlog, trfl_eventlog, nsml_eventlog)

In [19]:
# Function to create and transform a sequence into a numeric numpy array
def transform_sequence(ppid, date_req, nimess_eventlog, anino_eventlog, trfl_eventlog, encoded_nsml_eventlog):
    # Assuming create_sequences function is already defined and it combines the data correctly
    sequence, feature_names = create_sequences(ppid, date_req, nimess_eventlog, anino_eventlog, trfl_eventlog, encoded_nsml_eventlog)

    # Convert dataframe to numpy array
    numeric_array = sequence.to_numpy()
    return numeric_array, feature_names

In [20]:
def create_sequences(patient_id, test_date, nimess_eventlog, anino_eventlog, trfl_eventlog, nsml_eventlog, use_full_history=False, window_size=1095):
    # Define the start date for the sequence
    if use_full_history:
        start_date = pd.Timestamp.min
    else:
        start_date = test_date - pd.Timedelta(days=window_size)

    # Filter each dataframe for the given patient and time window
    nsml_events = nsml_eventlog[(nsml_eventlog['ppid'] == patient_id) &
                                (nsml_eventlog['date_column'] < test_date) &
                                (nsml_eventlog['date_column'] >= start_date)]

    trfl_events = trfl_eventlog[(trfl_eventlog['ppid'] == patient_id) &
                                (trfl_eventlog['date_column'] <= test_date) &
                                (trfl_eventlog['date_column'] >= start_date)]

    nimess_events = nimess_eventlog[(nimess_eventlog['ppid'] == patient_id) &
                                    (nimess_eventlog['date_column'] <= test_date) &
                                    (nimess_eventlog['date_column'] >= start_date)]

    anino_events = anino_eventlog[(anino_eventlog['ppid'] == patient_id) &
                                  (anino_eventlog['date_column'] <= test_date) &
                                  (anino_eventlog['date_column'] >= start_date)]

    # Merge the dataframes using outer join
    merged_df = nsml_events.merge(trfl_events, on=['ppid', 'date_column'], how='outer')
    merged_df = merged_df.merge(nimess_events, on=['ppid', 'date_column'], how='outer')
    merged_df = merged_df.merge(anino_events, on=['ppid', 'date_column'], how='outer')

    # Sort by date_column
    merged_df = merged_df.sort_values(by='date_column')

    # Remove 'date_column' and 'ppid' columns
    merged_df = merged_df.drop(columns=['date_column', 'ppid'])

    feature_names = merged_df.columns.tolist()

    # Additional data processing can be done here

    return merged_df, feature_names

In [21]:
# Initialize empty lists for sequences and labels
sequences = []
labels = []
feature_names_list = []  # To store feature names
# Define batch size
batch_size = 10000  # Adjust this based on your dataset and available memory

In [22]:
def process_batch(batch_df):
    batch_data = batch_df.apply(
        lambda row: transform_sequence(row['ppid'], row['date_column'], nimess_eventlog, anino_eventlog, trfl_eventlog, encoded_nsml_eventlog),
        axis=1
    )

    sequences_batch, feature_names_batch = zip(*batch_data)

    labels_batch = batch_df['blood_culture_test_encoded'].tolist()

    # Ensure all feature names are the same across batches
    assert all(feature_names_batch[0] == fn for fn in feature_names_batch), "Inconsistent feature names across batches"

    return sequences_batch, labels_batch, feature_names_batch[0]  # Returning the first batch's feature names


# Process in batches
for start in range(0, len(final_df), batch_size):
    end = min(start + batch_size, len(final_df))
    batch_df = final_df.iloc[start:end]
    sequences_batch, labels_batch, feature_names = process_batch(batch_df)
    sequences.extend(sequences_batch)
    labels.extend(labels_batch)
    if not feature_names_list:  # If feature_names_list is empty, populate it
        feature_names_list.extend(feature_names)

    print(f"Processed batch {start//batch_size + 1}/{len(final_df)//batch_size + 1}")


Processed batch 1/7
Processed batch 2/7
Processed batch 3/7
Processed batch 4/7
Processed batch 5/7
Processed batch 6/7
Processed batch 7/7


In [23]:
# Determine the length to pad/truncate to based on the longest sequence
max_sequence_length = max(len(sequence) for sequence in sequences)
max_sequence_length

538

In [24]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [25]:
sequence_length = min(max_sequence_length, 200)  # Setting an upper limit

In [ ]:
# Pad or truncate sequences in batches
padded_sequences = []
padding_batch_size = 1000  # Adjust based on available memory

for start in range(0, len(sequences), padding_batch_size):
    end = min(start + padding_batch_size, len(sequences))
    sequences_batch = sequences[start:end]
    padded_batch = pad_sequences(sequences_batch, maxlen=sequence_length, padding='pre', truncating='pre', dtype='float32')
    padded_sequences.extend(padded_batch)

    print(f"Padding batch {start//padding_batch_size + 1}/{len(sequences)//padding_batch_size + 1}")

# Convert the padded sequences list to a numpy array
padded_sequences = np.array(padded_sequences)

# Convert labels to a numpy array
labels = np.array(labels)


In [27]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [28]:
# Calculate the sequence length (if not already defined)
sequence_length = padded_sequences.shape[1]

# Generate flattened feature names
flattened_feature_names = []
for t in range(sequence_length):
    for feature_name in feature_names_list:
        flattened_feature_names.append(f"{feature_name}_t{t+1}")

In [29]:
from sklearn.utils.class_weight import compute_class_weight

# Calculate class weights
class_weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weight_dict = dict(enumerate(class_weights))

print("Class weights:", class_weight_dict)


Class weights: {0: 0.5444107178347517, 1: 6.129271765663141}


In [30]:
from sklearn.model_selection import train_test_split

# Initial train-test split with stratified sampling
X_train, X_test, y_train, y_test = train_test_split(
    padded_sequences,
    labels,
    test_size=0.2,
    random_state=None,
    shuffle=False)  # Notice the 'stratify' parameter here

# Further split the training data into a smaller train set and a validation set, also with stratified sampling
X_train_trans, X_val_trans, y_train_trans, y_val_trans = train_test_split(
    X_train,
    y_train,
    test_size=0.15,
    random_state=None,
    shuffle=True)  # And also here

# Now, X_train_lstm, X_val_lstm, and X_test should have the same class proportions as the original labels array

In [31]:
from sklearn.preprocessing import StandardScaler

# Replace NaN values with 0
X_train_trans = np.nan_to_num(X_train_trans)
X_val_trans = np.nan_to_num(X_val_trans)
X_test = np.nan_to_num(X_test)

# Store original shapes for training, validation, and test sets
original_shape_train = X_train_trans.shape  # (num_samples, timesteps, features)
original_shape_val = X_val_trans.shape  # Validation set shape
original_shape_test = X_test.shape  # Test set shape

# Flatten the data for scaling (reshape to 2D)
X_train_reshaped = X_train_trans.reshape(-1, X_train_trans.shape[-1])
X_val_reshaped = X_val_trans.reshape(-1, X_val_trans.shape[-1])
X_test_reshaped = X_test.reshape(-1, X_test.shape[-1])

# Initialize a StandardScaler and fit it on X_train_reshaped
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_reshaped)

# Apply the same transformation to X_val_reshaped and X_test_reshaped
X_val_scaled = scaler.transform(X_val_reshaped)
X_test_scaled = scaler.transform(X_test_reshaped)

# Reshape back to original shape
X_train_trans = X_train_scaled.reshape(original_shape_train)
X_val_trans = X_val_scaled.reshape(original_shape_val)
X_test = X_test_scaled.reshape(original_shape_test)

In [32]:
print(X_train_trans.shape, y_train_trans.shape)

(40978, 200, 110) (40978,)


Adding anomaly scores for the transformer model

In [33]:
if np.isnan(X_train_trans).any():
    print("NaN values present in the original data")
else:
    print("No NaN values in the original data")

# Assuming X_train is your 3D training data (samples, timesteps, features)
# Flatten or aggregate your 3D data (X_train) into 2D
X_train_2d = np.mean(X_train, axis=1)  # Mean aggregation

# Verify that the transformation was successful
print(f"Shape of X_train_2d: {X_train_2d.shape}")
print(f"Shape of y_train: {y_train_trans.shape}")
# After transformation, check if NaN values are introduced
if np.isnan(X_train_2d).any():
    print("NaN values detected after transformation")

No NaN values in the original data
Shape of X_train_2d: (48210, 110)
Shape of y_train: (40978,)
NaN values detected after transformation


In [34]:
# Fill NaN values in the transformed data, if they exist
X_train_2d = np.nan_to_num(X_train_2d)

In [35]:
from sklearn.ensemble import IsolationForest

# Filter out the normal data for training the anomaly detection model
normal_data_2d = X_train_2d[y_train == 0]

# Train the Isolation Forest model
iso_forest = IsolationForest(n_estimators=100, random_state=42, contamination='auto')
iso_forest.fit(normal_data_2d)

IsolationForest(random_state=42)

In [36]:
# Transform each subset from 3D to 2D
X_train_trans_2d = np.mean(X_train_trans, axis=1)
X_val_trans_2d = np.mean(X_val_trans, axis=1)
X_test_2d = np.mean(X_test, axis=1)
# Fill NaN values in the transformed data, if they exist
X_train_trans_2d = np.nan_to_num(X_train_trans_2d)
X_val_trans_2d = np.nan_to_num(X_val_trans_2d)
X_test_2d = np.nan_to_num(X_test_2d)

# Compute anomaly scores for each subset
anomaly_scores_train_trans = iso_forest.decision_function(X_train_trans_2d)
anomaly_scores_val_trans = iso_forest.decision_function(X_val_trans_2d)
anomaly_scores_test = iso_forest.decision_function(X_test_2d)

# Normalize the anomaly scores
anomaly_scores_train_norm = np.interp(anomaly_scores_train_trans, (anomaly_scores_train_trans.min(), anomaly_scores_train_trans.max()), (0, 1))
anomaly_scores_val_norm = np.interp(anomaly_scores_val_trans, (anomaly_scores_val_trans.min(), anomaly_scores_val_trans.max()), (0, 1))
anomaly_scores_test_norm = np.interp(anomaly_scores_test, (anomaly_scores_test.min(), anomaly_scores_test.max()), (0, 1))


In [37]:
# Function to expand and append anomaly scores to original data
def append_anomaly_scores(original_data, scores):
    # Expanding the scores to match the original data shape
    scores_expanded = np.expand_dims(scores, axis=-1)  # Adding an extra dimension
    scores_expanded = np.repeat(scores_expanded, original_data.shape[1], axis=1)  # Repeat across timesteps
    scores_expanded = np.expand_dims(scores_expanded, axis=-1)  # Adding another dimension for features

    return np.concatenate((original_data, scores_expanded), axis=2)  # Concatenate along the feature axis

# Apply the function to each subset
X_train_trans_extended = append_anomaly_scores(X_train_trans, anomaly_scores_train_norm)
X_val_trans_extended = append_anomaly_scores(X_val_trans, anomaly_scores_val_norm)
X_test_extended = append_anomaly_scores(X_test, anomaly_scores_test_norm)

In [38]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, auc, precision_recall_curve

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_pred_class = (y_pred > 0.5).astype("int32")

    accuracy = accuracy_score(y_test, y_pred_class)
    precision = precision_score(y_test, y_pred_class)
    recall = recall_score(y_test, y_pred_class)
    f1 = f1_score(y_test, y_pred_class)
    roc_auc = roc_auc_score(y_test, y_pred)

    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_class).ravel()
    specificity = tn / (tn + fp)

    # AUPRC calculation
    precision_curve, recall_curve, _ = precision_recall_curve(y_test, y_pred)
    auprc = auc(recall_curve, precision_curve)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall (Sensitivity): {recall:.4f}")
    print(f"Specificity: {specificity:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"ROC AUC Score: {roc_auc:.4f}")
    print(f"AUPRC Score: {auprc:.4f}")

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, average_precision_score, confusion_matrix
import numpy as np

def evaluate_model2(model, X_test, y_test):
    # Predict the probabilities
    y_pred_prob = model.predict(X_test)

    # Convert probabilities to class predictions
    y_pred = np.argmax(y_pred_prob, axis=1)
    y_true = np.argmax(y_test, axis=1)

    # Calculate accuracy
    accuracy = accuracy_score(y_true, y_pred)

    # Calculate precision
    precision = precision_score(y_true, y_pred, average='binary')

    # Calculate recall (sensitivity)
    recall = recall_score(y_true, y_pred, average='binary')

    # Calculate specificity
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    specificity = tn / (tn + fp)

    # Calculate F1 score
    f1 = f1_score(y_true, y_pred, average='binary')

    # Calculate ROC AUC score
    roc_auc = roc_auc_score(y_test, y_pred_prob, multi_class='ovr')

    # Calculate AUPRC score
    auprc = average_precision_score(y_test, y_pred_prob)

    # Print the metrics
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall (Sensitivity): {recall:.4f}")
    print(f"Specificity: {specificity:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"ROC AUC Score: {roc_auc:.4f}")
    print(f"AUPRC Score: {auprc:.4f}")


In [39]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [40]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, LayerNormalization, Input
from tensorflow.keras.layers import MultiHeadAttention, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam

In [41]:
# Set the learning rate
learning_rate = 0.0001

# Transformer block as a function
def transformer_block(inputs, num_heads, ff_dim, dropout=0.2):
    # Multi-head attention
    attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=ff_dim)(inputs, inputs)

    # Skip connection and normalization
    attention_output = Dropout(dropout)(attention_output)
    out1 = LayerNormalization(epsilon=1e-6)(inputs + attention_output)

    # Feed-forward layer
    ffn_output = Dense(ff_dim, activation='relu')(out1)
    ffn_output = Dropout(dropout)(ffn_output)
    ffn_output = Dense(inputs.shape[-1])(ffn_output)

    # Second skip connection and normalization
    out2 = LayerNormalization(epsilon=1e-6)(out1 + ffn_output)

    return out2

# Model definition
input_shape = (X_train_trans_extended.shape[1], X_train_trans_extended.shape[2])
inputs = Input(shape=input_shape)
x = inputs

# Create Transformer layers
for _ in range(2):  # Number of Transformer blocks
    x = transformer_block(x, num_heads=4, ff_dim=512)

# Output layers
x = GlobalAveragePooling1D()(x)
x = Dropout(0.2)(x)
x = Dense(20, activation="relu")(x)
x = Dropout(0.1)(x)
outputs = Dense(1, activation="sigmoid")(x)

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    X_train_trans_extended, y_train_trans,
    epochs=10,
    batch_size=128,
    validation_data=(X_val_trans_extended, y_val_trans),
    class_weight=class_weight_dict
)

Epoch 1/10
321/321 [==============================] - 1410s 4s/step - loss: 0.6738 - accuracy: 0.5739 - val_loss: 0.5956 - val_accuracy: 0.7181
Epoch 2/10
321/321 [==============================] - 1396s 4s/step - loss: 0.6153 - accuracy: 0.6924 - val_loss: 0.5440 - val_accuracy: 0.7359
Epoch 3/10
321/321 [==============================] - 1397s 4s/step - loss: 0.5895 - accuracy: 0.7208 - val_loss: 0.5803 - val_accuracy: 0.7439
Epoch 4/10
321/321 [==============================] - 1393s 4s/step - loss: 0.5722 - accuracy: 0.7486 - val_loss: 0.5354 - val_accuracy: 0.7931
Epoch 5/10
321/321 [==============================] - 1390s 4s/step - loss: 0.5550 - accuracy: 0.7735 - val_loss: 0.5432 - val_accuracy: 0.7884
Epoch 6/10
321/321 [==============================] - 1390s 4s/step - loss: 0.5451 - accuracy: 0.7813 - val_loss: 0.5671 - val_accuracy: 0.7811
Epoch 7/10
321/321 [==============================] - 1390s 4s/step - loss: 0.5347 - accuracy: 0.7857 - val_loss: 0.5907 - val_accuracy:

In [43]:
print("Evaluating Transformer Model")
evaluate_model(model, X_test_extended, y_test)
#print("Evaluating Transformer Model")
#evaluate_model2(model, X_test_extended, y_test)

Evaluating Transformer Model
377/377 [==============================] - 172s 456ms/step
Accuracy: 0.8167
Precision: 0.2339
Recall (Sensitivity): 0.5420
Specificity: 0.8413
F1 Score: 0.3267
ROC AUC Score: 0.7643
AUPRC Score: 0.2911


In [ ]:
def transformer_block(inputs, num_heads, ff_dim, dropout=0.1):
    # Multi-head attention
    attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=ff_dim)(inputs, inputs)

    # Skip connection and normalization
    attention_output = Dropout(dropout)(attention_output)
    out1 = LayerNormalization(epsilon=1e-6)(inputs + attention_output)

    # Feed-forward layer
    ffn_output = Dense(ff_dim, activation='relu')(out1)
    ffn_output = Dropout(dropout)(ffn_output)
    ffn_output = Dense(inputs.shape[-1])(ffn_output)

    # Second skip connection and normalization
    out2 = LayerNormalization(epsilon=1e-6)(out1 + ffn_output)

    return out2


# Assuming X and y contain your concatenated training and validation data
X = np.concatenate((X_train_trans_extended, X_val_trans_extended), axis=0)
y = np.concatenate((y_train_trans_, y_val_trans), axis=0)

# Define K-fold cross validator
k = 5  # Number of folds
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Define a function to create a new instance of your model
def create_model(input_shape):
    inputs = Input(shape=input_shape)
    x = inputs

    # Create Transformer layers
    for _ in range(2):  # Number of Transformer blocks
        x = transformer_block(x, num_heads=4, ff_dim=512)

    # Output layers
    x = GlobalAveragePooling1D()(x)
    x = Dropout(0.1)(x)
    x = Dense(20, activation="relu")(x)
    x = Dropout(0.1)(x)
    outputs = Dense(1, activation="sigmoid")(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Iterate over each fold
fold_no = 1
for train_index, val_index in kf.split(X):
    # Split data into training and validation for current fold
    X_train_fold, X_val_fold = X[train_index], X[val_index]
    y_train_fold, y_val_fold = y[train_index], y[val_index]

    # Create a new model instance for each fold
    model = create_model(input_shape=X_train_lstm.shape[1:])

    # Compile the model
    optimizer = Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model
    print(f'Training for fold {fold_no} ...')
    history = model.fit(
        X_train_fold, y_train_fold,
        epochs=50,
        batch_size=256,
        validation_data=(X_val_fold, y_val_fold),
        class_weight=class_weight_dict
    )

    # Increase fold number
    fold_no += 1


In [ ]:
print("Evaluating Transformer Model")
evaluate_model(model, X_test_extended, y_test)

In [44]:
from tensorflow.keras.layers import GRU, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten
# CNN-GRU Model
input_shape = (X_train_trans_extended.shape[1], X_train_trans_extended.shape[2])
inputs = Input(shape=input_shape)
x = Conv1D(filters=64, kernel_size=3, activation='relu')(inputs)
x = MaxPooling1D(pool_size=2)(x)
x = GRU(64)(x)
x = Dense(20, activation='relu')(x)
outputs = Dense(1, activation='sigmoid')(x)

cnn_gru_model = Model(inputs=inputs, outputs=outputs)
cnn_gru_model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
cnn_gru_model.fit(X_train_trans_extended, y_train_trans, epochs=50, batch_size=128, validation_data=(X_val_trans_extended, y_val_trans), class_weight=class_weight_dict)


Epoch 1/50
321/321 [==============================] - 29s 83ms/step - loss: 0.6707 - accuracy: 0.5686 - val_loss: 0.6217 - val_accuracy: 0.6961
Epoch 2/50
321/321 [==============================] - 25s 79ms/step - loss: 0.6060 - accuracy: 0.7045 - val_loss: 0.5857 - val_accuracy: 0.7297
Epoch 3/50
321/321 [==============================] - 25s 77ms/step - loss: 0.5690 - accuracy: 0.7451 - val_loss: 0.5646 - val_accuracy: 0.7449
Epoch 4/50
321/321 [==============================] - 25s 77ms/step - loss: 0.5449 - accuracy: 0.7593 - val_loss: 0.5635 - val_accuracy: 0.7431
Epoch 5/50
321/321 [==============================] - 25s 78ms/step - loss: 0.5254 - accuracy: 0.7722 - val_loss: 0.5268 - val_accuracy: 0.7700
Epoch 6/50
321/321 [==============================] - 25s 78ms/step - loss: 0.5075 - accuracy: 0.7822 - val_loss: 0.5313 - val_accuracy: 0.7624
Epoch 7/50
321/321 [==============================] - 25s 79ms/step - loss: 0.4907 - accuracy: 0.7903 - val_loss: 0.5498 - val_accuracy:

In [46]:
print("Evaluating CNN-GRU Model")
evaluate_model(cnn_gru_model, X_test_extended, y_test)

Evaluating CNN-GRU Model
377/377 [==============================] - 4s 11ms/step
Accuracy: 0.8612
Precision: 0.2425
Recall (Sensitivity): 0.3256
Specificity: 0.9091
F1 Score: 0.2779
ROC AUC Score: 0.6973
AUPRC Score: 0.2135


In [45]:
print("Evaluating CNN-GRU Model")
evaluate_model(cnn_gru_model, X_test_extended, y_test)

Evaluating CNN-GRU Model
377/377 [==============================] - 5s 11ms/step
Accuracy: 0.8612
Precision: 0.2425
Recall (Sensitivity): 0.3256
Specificity: 0.9091
F1 Score: 0.2779
ROC AUC Score: 0.6973
AUPRC Score: 0.2135


In [47]:
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten

# CNN-LSTM Model
input_shape = (X_train_trans_extended.shape[1], X_train_trans_extended.shape[2])
inputs = Input(shape=input_shape)
x = Conv1D(filters=64, kernel_size=3, activation='relu')(inputs)
x = MaxPooling1D(pool_size=2)(x)
x = LSTM(64)(x)
x = Dense(20, activation='relu')(x)
outputs = Dense(1, activation='sigmoid')(x)

cnn_lstm_model = Model(inputs=inputs, outputs=outputs)
cnn_lstm_model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
cnn_lstm_model.fit(X_train_trans_extended, y_train_trans, epochs=10, batch_size=128, validation_data=(X_val_trans_extended, y_val_trans), class_weight=class_weight_dict)


Epoch 1/10
321/321 [==============================] - 32s 94ms/step - loss: 0.6676 - accuracy: 0.6213 - val_loss: 0.6466 - val_accuracy: 0.6734
Epoch 2/10
321/321 [==============================] - 29s 89ms/step - loss: 0.6059 - accuracy: 0.7183 - val_loss: 0.5904 - val_accuracy: 0.7251
Epoch 3/10
321/321 [==============================] - 29s 90ms/step - loss: 0.5674 - accuracy: 0.7457 - val_loss: 0.5354 - val_accuracy: 0.7676
Epoch 4/10
321/321 [==============================] - 29s 89ms/step - loss: 0.5445 - accuracy: 0.7603 - val_loss: 0.5432 - val_accuracy: 0.7568
Epoch 5/10
321/321 [==============================] - 28s 89ms/step - loss: 0.5247 - accuracy: 0.7698 - val_loss: 0.5422 - val_accuracy: 0.7544
Epoch 6/10
321/321 [==============================] - 29s 89ms/step - loss: 0.5077 - accuracy: 0.7756 - val_loss: 0.5314 - val_accuracy: 0.7558
Epoch 7/10
321/321 [==============================] - 29s 90ms/step - loss: 0.4914 - accuracy: 0.7835 - val_loss: 0.5046 - val_accuracy:

In [48]:
print("Evaluating CNN-LSTM Model")
evaluate_model(cnn_lstm_model, X_test_extended, y_test)

Evaluating CNN-LSTM Model
377/377 [==============================] - 6s 14ms/step
Accuracy: 0.7079
Precision: 0.1732
Recall (Sensitivity): 0.6785
Specificity: 0.7105
F1 Score: 0.2760
ROC AUC Score: 0.7600
AUPRC Score: 0.3115


In [49]:
from tensorflow.keras.layers import Input, GRU, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# GRU Model
input_shape = (X_train_trans_extended.shape[1], X_train_trans_extended.shape[2])
inputs = Input(shape=input_shape)
x = GRU(128, return_sequences=True)(inputs)
x = GRU(64)(x)
x = Dense(20, activation='relu')(x)
outputs = Dense(1, activation='sigmoid')(x)

gru_model = Model(inputs=inputs, outputs=outputs)
gru_model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

gru_model.fit(X_train_trans_extended, y_train_trans, epochs=10, batch_size=128, validation_data=(X_val_trans_extended, y_val_trans), class_weight=class_weight_dict)


Epoch 1/10
321/321 [==============================] - 133s 402ms/step - loss: 0.6491 - accuracy: 0.5830 - val_loss: 0.6406 - val_accuracy: 0.6990
Epoch 2/10
321/321 [==============================] - 127s 397ms/step - loss: 0.5822 - accuracy: 0.7413 - val_loss: 0.5596 - val_accuracy: 0.7631
Epoch 3/10
321/321 [==============================] - 128s 399ms/step - loss: 0.5504 - accuracy: 0.7646 - val_loss: 0.5506 - val_accuracy: 0.7590
Epoch 4/10
321/321 [==============================] - 128s 398ms/step - loss: 0.5259 - accuracy: 0.7768 - val_loss: 0.4929 - val_accuracy: 0.7931
Epoch 5/10
321/321 [==============================] - 128s 399ms/step - loss: 0.5050 - accuracy: 0.7881 - val_loss: 0.5132 - val_accuracy: 0.7757
Epoch 6/10
321/321 [==============================] - 128s 398ms/step - loss: 0.4841 - accuracy: 0.7988 - val_loss: 0.4941 - val_accuracy: 0.7835
Epoch 7/10
321/321 [==============================] - 128s 400ms/step - loss: 0.4659 - accuracy: 0.8053 - val_loss: 0.4962 -

In [50]:
print("Evaluating GRU Model")
evaluate_model(gru_model, X_test_extended, y_test)

Evaluating GRU Model
377/377 [==============================] - 19s 48ms/step
Accuracy: 0.7835
Precision: 0.2193
Recall (Sensitivity): 0.6400
Specificity: 0.7964
F1 Score: 0.3267
ROC AUC Score: 0.7830
AUPRC Score: 0.3560


In [51]:
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# LSTM Model
input_shape = (X_train_trans_extended.shape[1], X_train_trans_extended.shape[2])
inputs = Input(shape=input_shape)
x = LSTM(128, return_sequences=True)(inputs)
x = LSTM(64)(x)
x = Dense(20, activation='relu')(x)
outputs = Dense(1, activation='sigmoid')(x)

lstm_model = Model(inputs=inputs, outputs=outputs)
lstm_model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
lstm_model.fit(X_train_trans_extended, y_train_trans, epochs=10, batch_size=128, validation_data=(X_val_trans_extended, y_val_trans), class_weight=class_weight_dict)


Epoch 1/10
321/321 [==============================] - 157s 474ms/step - loss: 0.6685 - accuracy: 0.7628 - val_loss: 0.6190 - val_accuracy: 0.6782
Epoch 2/10
321/321 [==============================] - 151s 471ms/step - loss: 0.6105 - accuracy: 0.6970 - val_loss: 0.5519 - val_accuracy: 0.7518
Epoch 3/10
321/321 [==============================] - 150s 468ms/step - loss: 0.5649 - accuracy: 0.7331 - val_loss: 0.5791 - val_accuracy: 0.7208
Epoch 4/10
321/321 [==============================] - 151s 469ms/step - loss: 0.5259 - accuracy: 0.7590 - val_loss: 0.6006 - val_accuracy: 0.6940
Epoch 5/10
321/321 [==============================] - 150s 468ms/step - loss: 0.4928 - accuracy: 0.7770 - val_loss: 0.5319 - val_accuracy: 0.7489
Epoch 6/10
321/321 [==============================] - 151s 469ms/step - loss: 0.4592 - accuracy: 0.7946 - val_loss: 0.4501 - val_accuracy: 0.7999
Epoch 7/10
321/321 [==============================] - 150s 468ms/step - loss: 0.4264 - accuracy: 0.8108 - val_loss: 0.5107 -

In [52]:
print("Evaluating LSTM Model")
evaluate_model(lstm_model, X_test_extended, y_test)

Evaluating LSTM Model
377/377 [==============================] - 31s 81ms/step
Accuracy: 0.7812
Precision: 0.2008
Recall (Sensitivity): 0.5592
Specificity: 0.8011
F1 Score: 0.2955
ROC AUC Score: 0.7568
AUPRC Score: 0.3186


In [53]:
from tensorflow.keras.utils import to_categorical

# Ensure the labels are one-hot encoded
y_train_trans = to_categorical(y_train_trans, num_classes=2)
y_val_trans = to_categorical(y_val_trans, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

# Now, define and train the DKN model again
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM, Flatten
from tensorflow.keras.models import Model

# Define the DKN model
def create_dkn_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    # LSTM layer
    lstm_out = LSTM(128, return_sequences=True)(inputs)

    # Flatten the LSTM output
    lstm_out_flattened = Flatten()(lstm_out)

    # Dense layers for knowledge-aware processing
    dense_out = Dense(128, activation='relu')(lstm_out_flattened)

    # Output layer
    outputs = Dense(num_classes, activation='softmax')(dense_out)

    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Define the input shape and number of classes
input_shape = (X_train_trans_extended.shape[1], X_train_trans_extended.shape[2])
num_classes = 2  # Assuming binary classification

# Create the model
dkn_model = create_dkn_model(input_shape, num_classes)

# Train the model
dkn_model.fit(X_train_trans_extended, y_train_trans, epochs=10, batch_size=32, validation_data=(X_val_trans_extended, y_val_trans))


Epoch 1/10
1281/1281 [==============================] - 200s 154ms/step - loss: 0.2602 - accuracy: 0.9196 - val_loss: 0.2379 - val_accuracy: 0.9219
Epoch 2/10
1281/1281 [==============================] - 196s 153ms/step - loss: 0.2082 - accuracy: 0.9287 - val_loss: 0.2375 - val_accuracy: 0.9235
Epoch 3/10
1281/1281 [==============================] - 196s 153ms/step - loss: 0.1704 - accuracy: 0.9387 - val_loss: 0.2695 - val_accuracy: 0.9205
Epoch 4/10
1281/1281 [==============================] - 196s 153ms/step - loss: 0.1276 - accuracy: 0.9542 - val_loss: 0.3014 - val_accuracy: 0.9098
Epoch 5/10
1281/1281 [==============================] - 196s 153ms/step - loss: 0.0894 - accuracy: 0.9686 - val_loss: 0.4037 - val_accuracy: 0.9129
Epoch 6/10
1281/1281 [==============================] - 196s 153ms/step - loss: 0.0688 - accuracy: 0.9756 - val_loss: 0.4546 - val_accuracy: 0.9115
Epoch 7/10
1281/1281 [==============================] - 196s 153ms/step - loss: 0.0464 - accuracy: 0.9839 - val_

In [54]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, average_precision_score, confusion_matrix
import numpy as np

def evaluate_model2(model, X_test, y_test):
    # Predict the probabilities
    y_pred_prob = model.predict(X_test)

    # Convert probabilities to class predictions
    y_pred = np.argmax(y_pred_prob, axis=1)
    y_true = np.argmax(y_test, axis=1)

    # Calculate accuracy
    accuracy = accuracy_score(y_true, y_pred)

    # Calculate precision
    precision = precision_score(y_true, y_pred, average='binary')

    # Calculate recall (sensitivity)
    recall = recall_score(y_true, y_pred, average='binary')

    # Calculate specificity
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    specificity = tn / (tn + fp)

    # Calculate F1 score
    f1 = f1_score(y_true, y_pred, average='binary')

    # Calculate ROC AUC score
    roc_auc = roc_auc_score(y_test, y_pred_prob, multi_class='ovr')

    # Calculate AUPRC score
    auprc = average_precision_score(y_test, y_pred_prob)

    # Print the metrics
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall (Sensitivity): {recall:.4f}")
    print(f"Specificity: {specificity:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"ROC AUC Score: {roc_auc:.4f}")
    print(f"AUPRC Score: {auprc:.4f}")


In [55]:
# Evaluate the model
evaluate_model2(dkn_model, X_test_extended, y_test)

377/377 [==============================] - 23s 61ms/step
Accuracy: 0.9012
Precision: 0.3412
Recall (Sensitivity): 0.2194
Specificity: 0.9621
F1 Score: 0.2671
ROC AUC Score: 0.6911
AUPRC Score: 0.6000


In [ ]:

# Evaluate the model
evaluate_model2(dkn_model, X_test_extended, y_test)


384/384 [==============================] - 23s 60ms/step
Accuracy: 0.9050
Precision: 0.3445
Recall (Sensitivity): 0.1972
Specificity: 0.9671
F1 Score: 0.2508
ROC AUC Score: 0.6909
AUPRC Score: 0.5901


In [57]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LayerNormalization, MultiHeadAttention, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define the CapMatch model
def create_capmatch_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    # Transformer block
    attn_output = MultiHeadAttention(num_heads=8, key_dim=input_shape[1])(inputs, inputs)
    attn_output = LayerNormalization(epsilon=1e-6)(attn_output)
    attn_output = Dropout(0.1)(attn_output)

    # Flatten the attention output
    attn_output_flattened = Flatten()(attn_output)

    # Capsule network block (simplified version)
    caps_output = Dense(128, activation='relu')(attn_output_flattened)

    # Output layer
    outputs = Dense(num_classes, activation='softmax')(caps_output)

    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Assuming input_shape is (timesteps, features)
input_shape = (X_train_trans_extended.shape[1], X_train_trans_extended.shape[2])
num_classes = 2  # Assuming binary classification

# Create the model
capmatch_model = create_capmatch_model(input_shape, num_classes)

# Train the model
capmatch_model.fit(X_train_trans_extended, y_train_trans, epochs=10, batch_size=32, validation_data=(X_val_trans_extended, y_val_trans))

# Evaluate the model
evaluate_model(capmatch_model, X_test_extended, y_test)


Epoch 1/10
1281/1281 [==============================] - 329s 256ms/step - loss: 0.5580 - accuracy: 0.9166 - val_loss: 0.2748 - val_accuracy: 0.9155
Epoch 2/10
1281/1281 [==============================] - 332s 259ms/step - loss: 0.2690 - accuracy: 0.9190 - val_loss: 0.3309 - val_accuracy: 0.9152
Epoch 3/10
1281/1281 [==============================] - 333s 260ms/step - loss: 0.2597 - accuracy: 0.9190 - val_loss: 0.2801 - val_accuracy: 0.9157
Epoch 4/10
1281/1281 [==============================] - 331s 259ms/step - loss: 0.2470 - accuracy: 0.9189 - val_loss: 0.3238 - val_accuracy: 0.9152
Epoch 5/10
1281/1281 [==============================] - 332s 260ms/step - loss: 0.2431 - accuracy: 0.9191 - val_loss: 0.2676 - val_accuracy: 0.9152
Epoch 6/10
1281/1281 [==============================] - 332s 259ms/step - loss: 0.2659 - accuracy: 0.9186 - val_loss: 0.2598 - val_accuracy: 0.9150
Epoch 7/10
1281/1281 [==============================] - 333s 260ms/step - loss: 0.3026 - accuracy: 0.9178 - val_

ValueError: Target is multilabel-indicator but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted', 'samples'].

In [58]:
evaluate_model2(capmatch_model, X_test_extended, y_test)

377/377 [==============================] - 42s 110ms/step
Accuracy: 0.9178
Precision: 0.0000
Recall (Sensitivity): 0.0000
Specificity: 0.9998
F1 Score: 0.0000
ROC AUC Score: 0.7631
AUPRC Score: 0.6214


In [59]:
from sklearn.utils import class_weight

# Compute class weights
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train_trans.argmax(axis=1)), y=y_train_trans.argmax(axis=1))
class_weights_dict = dict(enumerate(class_weights))

# Train the model with class weights
capmatch_model.fit(X_train_trans_extended, y_train_trans, epochs=10, batch_size=32, validation_data=(X_val_trans_extended, y_val_trans), class_weight=class_weights_dict)


Epoch 1/10
1281/1281 [==============================] - 330s 257ms/step - loss: 0.7488 - accuracy: 0.5195 - val_loss: 0.6964 - val_accuracy: 0.0848
Epoch 2/10
1281/1281 [==============================] - 328s 256ms/step - loss: 0.6940 - accuracy: 0.4406 - val_loss: 0.6884 - val_accuracy: 0.9155
Epoch 3/10
1281/1281 [==============================] - 327s 255ms/step - loss: 0.6932 - accuracy: 0.4605 - val_loss: 0.6898 - val_accuracy: 0.9155
Epoch 4/10
1281/1281 [==============================] - 327s 255ms/step - loss: 0.6933 - accuracy: 0.5905 - val_loss: 0.6918 - val_accuracy: 0.9155
Epoch 5/10
1281/1281 [==============================] - 326s 255ms/step - loss: 0.6933 - accuracy: 0.3818 - val_loss: 0.6926 - val_accuracy: 0.9155
Epoch 6/10
1281/1281 [==============================] - 325s 254ms/step - loss: 0.7525 - accuracy: 0.2702 - val_loss: 0.9220 - val_accuracy: 0.0868
Epoch 7/10
1281/1281 [==============================] - 327s 256ms/step - loss: 1.1106 - accuracy: 0.3064 - val_

In [60]:
evaluate_model2(capmatch_model, X_test_extended, y_test)

377/377 [==============================] - 41s 109ms/step
Accuracy: 0.0824
Precision: 0.0821
Recall (Sensitivity): 1.0000
Specificity: 0.0004
F1 Score: 0.1517
ROC AUC Score: 0.5003
AUPRC Score: 0.5002


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LayerNormalization, MultiHeadAttention, Dropout, Flatten, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define the CapMatch model
def create_capmatch_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    # Add a dimension to the input if necessary
    if len(input_shape) == 2:
        inputs_reshaped = Reshape((input_shape[0], input_shape[1], 1))(inputs)
    else:
        inputs_reshaped = inputs

    # Transformer block
    attn_output = MultiHeadAttention(num_heads=8, key_dim=input_shape[1])(inputs_reshaped, inputs_reshaped)
    attn_output = LayerNormalization(epsilon=1e-6)(attn_output)
    attn_output = Dropout(0.1)(attn_output)

    # Flatten the attention output
    attn_output_flattened = Flatten()(attn_output)

    # Capsule network block (simplified version)
    caps_output = Dense(128, activation='relu')(attn_output_flattened)

    # Output layer
    outputs = Dense(num_classes, activation='softmax')(caps_output)

    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Define the input shape and number of classes
input_shape = (X_train_trans_extended.shape[1], X_train_trans_extended.shape[2])
num_classes = 2  # Assuming binary classification

# Create the model
capmatch_model = create_capmatch_model(input_shape, num_classes)

# Train the model
capmatch_model.fit(X_train_trans_extended, y_train_trans, epochs=10, batch_size=32, validation_data=(X_val_trans_extended, y_val_trans))


Epoch 1/10


In [ ]:
evaluate_model2(capmatch_model, X_test_extended, y_test)